## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
import os

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(["NAME", "EIN"], axis=1, inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = []

# https://stackoverflow.com/questions/44353516/get-keys-of-pandas-series-value-counts
# adds all keys with a value of less than 10 to the application_types_to_replace list
for key, val in application_df["APPLICATION_TYPE"].value_counts().to_dict().items():
    if val < 200:
        application_types_to_replace.append(key)
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [4]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []

for key, val in application_df["CLASSIFICATION"].value_counts().to_dict().items():
    if val < 1000:
        classifications_to_replace.append(key)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [5]:
# cutting off AFFILIATION values with less than 100 entries
affiliations_to_replace = []

for key, val in application_df["AFFILIATION"].value_counts().to_dict().items():
    if val < 1000:
        classifications_to_replace.append(key)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(cls,"Other")

In [6]:
# cutting off USE_CASE values with less than 500 entries
use_cases_to_replace = []

for key, val in application_df["USE_CASE"].value_counts().to_dict().items():
    if val < 500:
        use_cases_to_replace.append(key)

# Replace in dataframe
for cls in use_cases_to_replace:
    application_df['USE_CASE'] = application_df['USE_CASE'].replace(cls,"Other")

In [7]:
# convert the income amount to income bracket that is represented by the middle value in the corresponding ranges
incomes = {
    "0": 0,
    "1-9999": 5000,
    "10000-24999": 17500,
    "100000-499999": 300000,
    "10M-50M": 30000000,
    "25000-99999": 62500,
    "50M+": 50000000,
    "1M-5M": 3000000,
    "5M-10M": 7500000
}
application_df["INCOME_AMT"] = application_df["INCOME_AMT"].map(lambda x: incomes[x])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,5000,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,17500,N,6692,1
4,T3,Independent,C1000,Other,Trust,1,300000,N,142590,1


In [8]:
# drop special considerations column
application_df.drop(["SPECIAL_CONSIDERATIONS"], axis=1, inplace=True)

# drop rows where STATUS is 0 and drop STATUS column
application_df = application_df.loc[application_df["STATUS"] == 1]
application_df.drop(["STATUS"], axis=1, inplace=True)


In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
# APPLICATION_TYPE, AFFILIATION, CLASSIFICATION, USE_CASE, ORGANIZATION
applictation_df_dummy = pd.get_dummies(application_df, columns=["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION"], dtype="int64")
applictation_df_dummy.head()

,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,0,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
1,5000,108590,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,0,5000,0,0,0,0,0,0,1,0,...,0,1,0,0,0,1,1,0,0,0
3,17500,6692,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
4,300000,142590,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [10]:
# Split our preprocessed data into our features and target arrays
y = applictation_df_dummy["IS_SUCCESSFUL"]
X = applictation_df_dummy.drop(["IS_SUCCESSFUL"], axis=1)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Optimizing the Model using HyperParameters

In [12]:
def create_model(hp):
    # create model
    nn = tf.keras.models.Sequential()

    # create list containing different activation functions
    activation = hp.Choice("activation", ["relu", "tanh"])

    # add first layer
    nn.add(tf.keras.layers.Dense(units=hp.Int("first_units",
                                              min_value=16,
                                              max_value=256,
                                              step=32),
                                              activation=activation,
                                              input_dim=len(X.columns)))
    
    # add random amount of hidden layers
    for i in range(hp.Int("num_layers", 2, 4)):
        nn.add(tf.keras.layers.Dense(units=hp.Int("units_" + str(i),
                                                  min_value=16,
                                                  max_value=128,
                                                  step=16),
                                                  activation=activation))
        
    # add output layer
    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # compile model
    nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

    # return model
    return nn
    

In [13]:
# create tuner
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=100,
)

# run tuner
tuner.search(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test))

Reloading Tuner from .\untitled_project\tuner0.json


In [14]:
# find the best model
best_model = tuner.get_best_models(1)[0]

# get and print its loss and accuracy
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

c:\Users\Paul\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Paul\anaconda3\envs\dev\lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


268/268 - 1s - 2ms/step - accuracy: 0.7435 - loss: 0.5386
Loss: 0.5385536551475525, Accuracy: 0.7435269355773926


In [15]:
# Export our model to HDF5 file
best_model.save("AlphabetSoupCharity_Best.h5")